In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def minkowski_distance(col1, col2, p):
    return (((abs(col1-col2))**p).sum())**(1/p)

In [ ]:
df=pd.read_csv("/content/Data.csv")

In [ ]:
df

,Serial,Sen1,Sen2,Score,Type
0,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
1,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
2,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
3,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL
4,12,Two dogs are fighting,Two dogs are wrestling and hugging,4.0,NEUTRAL
...,...,...,...,...,...
1270,3516,The man is not dancing,The man is dancing,3.9,CONTRADICTION
1271,3517,The man is wildly dancing,A woman is dancing,3.0,NEUTRAL
1272,3528,The person is not writing with a pencil,A person is writing,3.4,NEUTRAL
1273,3529,The writing is about a person penciling,A person is writing,2.9,NEUTRAL


In [ ]:
df['Score'].mask(df['Type'] == 'CONTRADICTION', 5-df['Score'], inplace=True)

In [ ]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
df["V1"]=0

In [ ]:
df["V2"]=0

In [ ]:
for i in range(len(df)):
  df["V1"][i]=[model.encode(df["Sen1"][i])]

<ipython-input-38-af504602345f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["V1"][i]=[model.encode(df["Sen1"][i])]


In [ ]:
for i in range(len(df)):
  df["V2"][i]=[model.encode(df["Sen2"][i])]

<ipython-input-41-a670abdcde5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["V2"][i]=[model.encode(df["Sen2"][i])]


In [ ]:
df['Man']=0
df['Euc']=0
df["Fra"]=0

In [ ]:
for i in range(len(df)):
  df['Man'][i]=minkowski_distance(df['V1'][i][0], df['V2'][i][0], 1)

<ipython-input-51-9ba5ce598e05>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Man'][i]=minkowski_distance(df['V1'][i][0], df['V2'][i][0], 1)


In [ ]:
for i in range(len(df)):
  df['Euc'][i]=minkowski_distance(df['V1'][i][0], df['V2'][i][0], 2)
for i in range(len(df)):
  df['Fra'][i]=minkowski_distance(df['V1'][i][0], df['V2'][i][0], 0.3)

<ipython-input-58-6d2a58dd3bfe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Euc'][i]=minkowski_distance(df['V1'][i][0], df['V2'][i][0], 2)
<ipython-input-58-6d2a58dd3bfe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fra'][i]=minkowski_distance(df['V1'][i][0], df['V2'][i][0], 0.3)


In [ ]:
def rescale(col):
  min_value = df[col].min()
  max_value = df[col].max()
  df[col] = 1 + ((df[col] - min_value) / (max_value - min_value)) * 4

In [ ]:
rescale("Man")
rescale("Euc")
rescale("Fra")

In [ ]:
df

,Serial,Sen1,Sen2,Score,Type,V1,V2,Man,Euc,Fra
0,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL,"[[0.47545886, -0.037648376, 0.08927051, -0.417...","[[0.49139023, 0.28649983, 0.16801298, -0.50579...",2.186306,2.218112,2.190842
1,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT,"[[-0.13462979, 0.49890336, -0.021877835, -0.85...","[[0.095526055, 0.49726602, 0.02861956, -0.6162...",2.041683,2.061883,2.008938
2,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL,"[[0.095526055, 0.49726602, 0.02861956, -0.6162...","[[0.49139023, 0.28649983, 0.16801298, -0.50579...",2.073540,2.127580,2.029087
3,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL,"[[-0.13462979, 0.49890336, -0.021877835, -0.85...","[[0.49139023, 0.28649983, 0.16801298, -0.50579...",2.569359,2.594489,2.530572
4,12,Two dogs are fighting,Two dogs are wrestling and hugging,4.0,NEUTRAL,"[[-0.5750578, -0.013041396, -0.101274185, 0.90...","[[-0.48545125, -0.22018507, 0.32654387, 0.5700...",2.810811,2.891492,2.736415
...,...,...,...,...,...,...,...,...,...,...
1270,3516,The man is not dancing,The man is dancing,1.1,CONTRADICTION,"[[-0.030196952, -0.07901152, 0.11262397, -0.53...","[[-0.056503493, -0.25535432, -0.19698422, -0.4...",3.123201,3.177171,3.072332
1271,3517,The man is wildly dancing,A woman is dancing,3.0,NEUTRAL,"[[0.12587568, 0.15739442, -0.25980294, -0.4809...","[[0.099282235, -0.69487065, 0.31376746, 0.0375...",3.526208,3.526939,3.521478
1272,3528,The person is not writing with a pencil,A person is writing,3.4,NEUTRAL,"[[-0.44194144, -0.33895132, -0.33055562, -0.27...","[[-0.5747519, -0.08423195, -0.31955644, 0.3946...",3.121074,3.140613,3.110786
1273,3529,The writing is about a person penciling,A person is writing,2.9,NEUTRAL,"[[-0.7019796, -0.121145844, -0.5826938, -0.034...","[[-0.5747519, -0.08423195, -0.31955644, 0.3946...",2.567321,2.628725,2.524810


In [ ]:
def loss(col):
  return abs(df["Score"]-df[col]).sum()/len(df)

In [ ]:
loss("Man")

1.5948092589433855

In [ ]:
loss("Euc")

1.5952337832536276

In [ ]:
loss("Fra")

1.5951396736361743